<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/PyTorch_%EC%8B%A4%EC%8A%B5%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch에서 데이터 다루기

## Dataset 살펴보기 : 간단한 데이터

### Dataset 클래스 만들기

In [1]:
import torch

Dataset 클래스는 데이터셋과 레이블을 관리하기 편리하게 만들어 주는 PyTorch의 유틸리티입니다. 커스텀 데이터셋을 정의하면, 데이터셋의 크기와 특정 인덱스 데이터를 쉽게 확인할 수 있습니다.

이번 예제에서는 간단한 2차원 데이터와 이진 레이블을 활용해 Dataset 클래스를 만들어보겠습니다.

In [2]:
# Dataset 클래스를 상속받아 커스텀 데이터셋을 정의합니다.
from torch.utils.data import Dataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x = [
            [10, 20],
            [20, 15],
            [15, 15],
            [80, 90],
            [85, 80],
            [85, 90],
        ]
        self.y = [0, 0, 0, 1, 1, 1]

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        input_data = self.x[index]
        label = self.y[index]
        return torch.tensor(input_data), torch.tensor(label)

사용자 정의 Dataset 클래스는 __init__ , __len__ , __getitem__ 의 세 가지 함수를 구현해야 합니다

- `__init__` 메소드는 클래스가 초기화될 때 실행됩니다. 데이터의 경로나 전처리를 정의하며, 간단한 데이터라면 여기에서 바로 데이터를 정의할 수도 있습니다.

- `__len__` 메소드는 데이터 셋에 있는 샘플의 개수를 반환합니다.

- `__getitem__` 메소드는 특정 인덱스의 데이터를 반환합니다.

### Dataset 객체 사용하기

In [4]:
### 객체 생성
custom_dataset = CustomDataset()

In [5]:
### 데이터 개수 확인
len(custom_dataset)

6

In [6]:
### 특정 인덱스 데이터 확인
custom_dataset[0]

(tensor([10, 20]), tensor(0))

In [7]:
custom_dataset[3]

(tensor([80, 90]), tensor(1))

## CSV 데이터 활용하기

 실제 데이터를 담고 있는 CSV 파일을 활용하여 PyTorch의 Dataset 클래스를 정의하고 사용하는 방법을 배워보겠습니다.

In [8]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset

전복(Abalone) 데이터셋을 판다스를 사용해 읽어옵니다.

이 데이터는 전복의 크기, 무게, 그리고 나이 정보를 포함하고 있습니다.

In [9]:
abalone_df = pd.read_csv(
    'https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv',
    names=['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
           'Viscera weight', 'Shell weight', 'Age']
)

In [10]:
abalone_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


결과를 보면, 길이, 직경, 높이 같은 크기 정보와 무게, 나이 정보가 잘 나옵니다.

이 데이터를 사용하여 전복의 나이를 예측하는 회귀 문제를 설정하겠습니다.

- **`input_data`**: 나이를 제외한 나머지 데이터 (입력)
- **`target_data`**: 전복의 나이 (타깃)

In [16]:
input_data = abalone_df.drop(columns=['Age']).to_numpy().astype(np.float32)
target_data = abalone_df['Age'].to_numpy().astype(np.float32)

In [12]:
input_data.shape

(3320, 7)

In [19]:
target_data.shape

(3320,)

In [27]:
input_data = abalone_df.drop(columns=['Age']).to_numpy().astype(np.float32)
target_data = abalone_df['Age'].to_numpy().astype(np.float32)

In [28]:
class AbaloneDataset(Dataset):
    def __init__(self, input_data, target_data):
        self.input_data = input_data
        self.target_data = target_data

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, index):
        input_tensor = torch.tensor(self.input_data[index])
        target_tensor = torch.tensor(self.target_data[index])

        return input_tensor, target_tensor

In [29]:
abalone_dataset = AbaloneDataset(input_data, target_data)

print(len(abalone_dataset))
print(abalone_dataset[0])

3320
(tensor([0.4350, 0.3350, 0.1100, 0.3340, 0.1355, 0.0775, 0.0965]), tensor(7.))


## 이미지 데이터

데이터 크기가 너무 커서 메모리에 모두 올리기 어려운 경우를 대비해 PyTorch Dataset 클래스를 활용하는 방법을 배워봅니다.

In [30]:
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

In [31]:
# 다양한 꽃 이미지 데이터가 담긴 flower_photos 폴더를 다운로드하고 압축을 해제합니다.

!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xvf ./flower_photos.tgz > /dev/null 2>&1


--2025-09-08 08:33:44--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 173.194.212.207, 74.125.139.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   167MB/s    in 1.3s    

2025-09-08 08:33:46 (167 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]



In [32]:
import glob
## glob 긁어서 list형태로 만들어놓기
image_path = glob.glob("/content/flower_photos/*/*.jpg")